In [1]:
import os
import numpy as np 
import pandas as pd
import arcpy
from arcpy import env

In [2]:
result_path = r"D:\ArcGISProjects\workspace\pred_feature_table\dy\predict_table\prediction_ph.csv"

In [3]:
# 读取预测结果
data = pd.read_csv(result_path,index_col=False)

In [4]:
stander_path = r"D:\ArcGISProjects\workspace\shbyq\features_data_dy.gdb\DEM"

In [5]:
data.columns

Index(['prediction_0', 'prediction_1', 'prediction_2', 'prediction_3',
       'prediction_4', 'prediction_5', 'prediction_6', 'prediction_7',
       'prediction_8', 'prediction_9',
       ...
       'prediction_334', 'prediction_335', 'prediction_336', 'prediction_337',
       'prediction_338', 'prediction_339', 'prediction_340', 'prediction_341',
       'prediction_342', 'prediction_343'],
      dtype='object', length=344)

In [6]:
# 合并除第一列外的所有列到第一列
frames = []
for col in data.columns:
    frames.append(data[col])

# 合并 DataFrame
merged_data = pd.concat(frames, axis=0, ignore_index=True)

In [8]:
merged_data.dropna(inplace=True)

In [9]:
merged_data.size

171871183

In [20]:
type(merged_data)

pandas.core.series.Series

In [10]:
dem_array = arcpy.RasterToNumPyArray(stander_path)
dem_array.shape

(12881, 13343)

In [11]:
dem_array.shape[0] * dem_array.shape[1]

171871183

In [12]:
raster_array = np.reshape(merged_data.values,dem_array.shape)
raster_array

array([[6.2363377, 6.2363377, 6.2363377, ..., 6.4920735, 6.4920735,
        6.4920735],
       [6.2363377, 6.2363377, 6.2363377, ..., 6.4920735, 6.4920735,
        6.4920735],
       [6.2363377, 6.2363377, 6.2363377, ..., 6.4920735, 6.4920735,
        6.4920735],
       ...,
       [5.999602 , 5.999602 , 5.999602 , ..., 5.8613887, 5.8613887,
        5.8613887],
       [5.999602 , 5.999602 , 5.999602 , ..., 5.8613887, 5.8613887,
        5.8613887],
       [5.999602 , 5.999602 , 5.999602 , ..., 5.8613887, 5.8613887,
        5.8613887]])

In [14]:
env.workspace = r"D:\ArcGISProjects\workspace\shbyq\feature_soiltype_result.gdb"
env.extent = stander_path

In [15]:
# 掩膜提取
def mask_raster(array,mask_ele,cell_size):
    out_raster = arcpy.NumPyArrayToRaster(
    array,
    arcpy.Point(arcpy.env.extent.XMin, arcpy.env.extent.YMin),
    cell_size,
    cell_size,
)
    """按掩膜提取栅格,空间参考设定为:CGCS2000_3_Degree_GK_CM_108E"""
    output_coordinate_system = arcpy.Describe(mask_ele).spatialReference
    with arcpy.EnvManager(outputCoordinateSystem=output_coordinate_system,snapRaster=mask_ele, cellSize=mask_ele):
        result_raster = arcpy.sa.ExtractByMask(out_raster, mask_ele, "INSIDE")
        return result_raster

In [17]:
# 按掩膜处理
result_raster = mask_raster(raster_array,stander_path, 5)
result_raster.save("prediction_ph")
print("完成")

完成
